In [60]:
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn import metrics
import keras
%matplotlib inline

traindata = pd.read_csv("train_values.csv",sep=',')
traindata.columns[traindata.isna().any()]
data=traindata
data.drop(['building_id'], axis=1, inplace=True)
trainlabels = pd.read_csv("train_labels.csv",sep=',')
trainout=trainlabels
trainout.drop(['building_id'], axis=1, inplace=True)
trainout.drop(['damage_grade'], axis=1, inplace=True)
testvalues = pd.read_csv("test_values.csv",sep=',')
testdata=testvalues
testdata.drop(['building_id'], axis=1, inplace=True)

In [61]:
cleanup_letters = {"land_surface_condition": {"n": 1, "o": 2, "t":3},
                "foundation_type": {"h": 1, "i": 2, "r": 3, "u": 4, "w": 5},               
                "roof_type": {"n": 1, "q": 2, "x": 3},
                "ground_floor_type": {"f": 1, "m": 2, "v": 3, "x": 4, "z": 5},
                "other_floor_type": {"j": 1, "q": 2, "s": 3, "x": 4},
                "position": {"j": 1, "o": 2, "s": 3, "t": 4},
                "plan_configuration": {"a": 1, "c": 2, "d": 3, "f": 4, "m": 5, "n": 6, "o": 7, "q": 8, "s": 9, "u":10},
                "legal_ownership_status": {"a": 1, "r": 2, "v": 3, "w": 4}             
               }
data.replace(cleanup_letters, inplace=True)
testdata.replace(cleanup_letters, inplace=True)
a = list(data)
del a[-1]
x = data[a].values


In [62]:

y = trainout['damage_grade_n'].values

y = keras.utils.to_categorical(y,3, dtype='int64')


In [ ]:
pd.unique(y)

In [63]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=21)

In [ ]:
pd.unique(y_train)

In [64]:
y_test

array([[0, 1, 0],
       [0, 0, 1],
       [0, 1, 0],
       ...,
       [0, 1, 0],
       [0, 0, 1],
       [0, 0, 1]], dtype=int64)

In [24]:
import keras
from keras.models import Sequential
from sklearn.neural_network import MLPClassifier
from keras.layers import Dense
classifier = Sequential()
# one rule to determine the number of hidden layers
# is to average the number of input and output nodes
classifier.add(Dense(units=37, input_dim=37, activation='relu'))
classifier.add(Dense(units=32, activation='tanh'))
classifier.add(Dense(units=32, activation='relu'))
classifier.add(Dense(units=20, activation='tanh'))
"""classifier.add(Dense(units=20, activation='relu'))
classifier.add(Dense(units=16, activation='tanh'))
classifier.add(Dense(units=16, activation='relu'))
classifier.add(Dense(units=8, activation='tanh'))
classifier.add(Dense(units=8, activation='relu'))
classifier.add(Dense(units=4, activation='relu'))"""
classifier.add(Dense(units=4, activation='tanh'))

classifier.add(Dense(units=4, activation='softmax'))
classifier.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [65]:
import tensorflow as tf
classifier = tf.keras.models.Sequential([
    tf.keras.layers.Flatten(input_shape=(37, )),
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dropout(0.05),
    tf.keras.layers.Dense(16, activation='tanh'),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(16, activation='relu'),
    tf.keras.layers.Dropout(0.1),
    tf.keras.layers.Dense(3, activation='softmax')
])

classifier.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [66]:
histor=classifier.fit(x_train, y_train, 
               epochs=10, batch_size=1024)#,validation_split=0.2)#,verbose=0)

Epoch 1/10
208480/208480 [==============================] - 1s 7us/sample - loss: 0.9450 - acc: 0.5573
Epoch 2/10
208480/208480 [==============================] - 1s 5us/sample - loss: 0.9236 - acc: 0.5679
Epoch 3/10
208480/208480 [==============================] - 1s 5us/sample - loss: 0.9192 - acc: 0.5686
Epoch 4/10
208480/208480 [==============================] - 1s 5us/sample - loss: 0.9176 - acc: 0.5686
Epoch 5/10
208480/208480 [==============================] - 1s 5us/sample - loss: 0.9166 - acc: 0.5686
Epoch 6/10
208480/208480 [==============================] - 1s 5us/sample - loss: 0.9157 - acc: 0.5687
Epoch 7/10
208480/208480 [==============================] - 1s 5us/sample - loss: 0.9153 - acc: 0.5687
Epoch 8/10
208480/208480 [==============================] - 1s 5us/sample - loss: 0.9147 - acc: 0.5687
Epoch 9/10
208480/208480 [==============================] - 1s 5us/sample - loss: 0.9146 - acc: 0.5687
Epoch 10/10
208480/208480 [==============================] - 1s 6us/sampl

In [67]:

print(history.summary())

AttributeError: 'History' object has no attribute 'summary'

In [67]:
import numpy as np
from sklearn.metrics import confusion_matrix
y_pred = classifier.predict(x_test)
y_pred = (y_pred > 0.5)
y_pred = np.argmax(y_pred, axis=1)
#confusion_matrix(y_test, y_pred)

In [68]:
y_test = (y_test> 0.5)
y_test = np.argmax(y_test, axis=1)

In [69]:
y_test

array([1, 2, 1, ..., 1, 2, 2], dtype=int64)

In [70]:
from sklearn import metrics
print(metrics.classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.00      0.00      0.00      4967
           1       0.57      1.00      0.73     29681
           2       0.00      0.00      0.00     17473

   micro avg       0.57      0.57      0.57     52121
   macro avg       0.19      0.33      0.24     52121
weighted avg       0.32      0.57      0.41     52121



C:\Users\HP\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [71]:
import sklearn
from sklearn import metrics
from sklearn.metrics import accuracy_score
acc=accuracy_score(y_test, y_pred, normalize=True, sample_weight=None)
print(acc)
y_pr=classifier.predict_proba(x_test)
from sklearn.metrics import log_loss
ll=sklearn.metrics.log_loss(y_test, y_pr, normalize=True)
print(ll)

0.5694633640950865
0.9114675386381845


In [72]:
from sklearn.metrics import f1_score
f1=f1_score(y_test,y_pred,average='micro')
f1

0.5694633640950865

In [74]:
from sklearn import preprocessing
def multiclass_roc_auc_score(y_test, y_pred, average="macro"):
 lb = preprocessing.LabelBinarizer()
 lb.fit(y_test)
 y_test = lb.transform(y_test)
 y_pred = lb.transform(y_pred)
 return roc_auc_score(y_test, y_pred, average=average)
from sklearn.metrics import roc_auc_score
roc=multiclass_roc_auc_score(y_test, y_pred, average="weighted")
print(roc)

0.5


In [31]:
pd.unique(y_pred)

array([2, 1, 0], dtype=int64)

In [32]:
y_pred=np.where(y_pred==0, 3, y_pred)

In [20]:
pd.unique(y_pred)

array([2, 3], dtype=int64)

In [21]:
from sklearn.metrics import f1_score
f1=f1_score(y_test,y_pred,average='micro')
f1

0.5698854588361697

In [172]:
y_pre=classifier.predict_classes(x_test)

In [173]:
pd.unique(y_pre)

array([2, 1], dtype=int64)

In [113]:
import matplotlib.pyplot as plt

acc = history.history['acc']
val_acc = history.history['val_acc']
loss = history.history['loss']
val_loss = history.history['val_loss']
epochs = range(1, len(acc) + 1)

plt.plot(epochs, acc, 'b', label='Training acc')
plt.plot(epochs, val_acc, 'g', label='Validation acc')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')

plt.title('Training and validation accuracy')
plt.legend()
fig = plt.figure()
fig.savefig('acc.png')


plt.plot(epochs, loss, 'b', label='Training loss')
plt.plot(epochs, val_loss, 'g', label='Validation loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.title('Training and validation loss')

plt.legend()
plt.show()

KeyError: 'val_acc'

In [26]:
model = Sequential()
model.add(Dense(input_dim=37, output_dim=512, init='glorot_uniform'))
model.add(PReLU(input_shape=(512,)))
model.add(BatchNormalization((512,)))
model.add(Dropout(0.5))

model.add(Dense(input_dim=512, output_dim=512, init='glorot_uniform'))
model.add(PReLU(input_shape=(512,)))
model.add(BatchNormalization((512,)))
model.add(Dropout(0.5))

model.add(Dense(input_dim=512, output_dim=512, init='glorot_uniform'))
model.add(PReLU(input_shape=(512,)))
model.add(BatchNormalization((512,)))
model.add(Dropout(0.5))

model.add(Dense(input_dim=512, output_dim=512, init='glorot_uniform'))
model.add(PReLU(input_shape=(512,)))
model.add(BatchNormalization((512,)))
model.add(Dropout(0.5))

model.add(Dense(input_dim=512, output_dim=512, init='glorot_uniform'))
model.add(PReLU(input_shape=(512,)))
model.add(BatchNormalization((512,)))
model.add(Dropout(0.5))

model.add(Dense(input_dim=512, output_dim=3, init='glorot_uniform'))
model.add(Activation('softmax'))

C:\Users\HP\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(input_dim=37, units=512, kernel_initializer="glorot_uniform")`
  


NameError: name 'PReLU' is not defined

In [ ]:
model.fit(x_train, y_train, 
               epochs=100, 
               batch_size=1000)

In [ ]:
score = model.evaluate(X_test, y_test, batch_size=16)

In [ ]:
from tensorflow.keras.layers import Dense, Embedding, LSTM
from tensorflow.keras.models import Sequential
model = Sequential()
model.add(Embedding(max_features, 32))
model.add(LSTM(32))
model.add(Dense(1, activation='sigmoid'))
model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['accuracy'])
history = model.fit(x_train, y_train, epochs=20, batch_size=128, validation_split=0.2)

In [ ]:
import matplotlib.pyplot as plt

acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']
epochs = range(1, len(acc) + 1)

plt.plot(epochs, acc, 'b', label='Training acc')
plt.plot(epochs, val_acc, 'g', label='Validation acc')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')

plt.title('Training and validation accuracy')
plt.legend()
fig = plt.figure()
fig.savefig('acc.png')


plt.plot(epochs, loss, 'b', label='Training loss')
plt.plot(epochs, val_loss, 'g', label='Validation loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.title('Training and validation loss')

plt.legend()
plt.show()